In [1]:
from taxcalc import *
from taxcalc.utils import *
from bokeh.io import show, output_notebook
from bkcharts import Bar, Scatter
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool
from collections import OrderedDict
import copy
from notebookfunctions import distribution, index_list, percentile
output_notebook()
import json

Loading BokehJS ...

In [2]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [3]:
# Create calculator from the PUF
calc = Calculator(records=Records(), policy=Policy())
calc.advance_to_year(2014)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [4]:
# Calculator the CPS
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights_lp_2.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios_lp.csv', index_col=0)
adj = adj.transpose()
recs_pulp = Records(weights=wt, adjust_ratios=adj)
pol_pulp = Policy()
calc_pulp = Calculator(records=recs_pulp, policy=pol_pulp)
calc_pulp.advance_to_year(2014)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
calc.calc_all()
calc_pulp.calc_all()

### PULP Distribution Table

In [6]:
create_distribution_table(calc_pulp.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-4,746","16,973,235","-5,298","12,022,440","6,334","11,580",17,"4,297",46,3,"-5,606",0,0,3,0,6,210,-201,495,295
1,"10,364","16,973,206","7,162","13,897,115","7,234","115,351",73,"5,286",543,52,"7,108","16,459",0,53,3,5,997,-942,"1,030",88
2,"17,023","16,973,328","11,552","14,224,789","7,657","356,602",238,"6,599","2,146",213,"11,399","29,802",1,214,34,5,"1,625","-1,440","1,656",216
3,"24,555","16,973,195","17,817","15,086,560","7,947","862,709",680,"7,045","5,404",593,"17,397","19,607",1,594,130,12,"1,399",-922,"2,475","1,553"
4,"33,154","16,974,227","25,383","14,691,307","8,012","1,887,858","1,530","7,401","10,613","1,245","24,460","14,932",2,"1,247",265,17,990,9,"3,332","3,341"
5,"43,553","16,973,446","35,607","13,677,627","7,737","3,219,555","2,899","7,567","18,653","2,297","33,906","10,827",2,"2,298",409,25,462,"1,453","4,492","5,945"
6,"56,828","16,973,010","48,891","11,729,867","7,161","5,225,571","5,213","7,963","28,979","3,831","45,818","10,607",1,"3,832",545,37,145,"3,179","5,805","8,985"
7,"76,445","16,973,877","69,734","9,507,089","6,260","7,461,253","8,168","8,817","46,594","6,716","65,230","40,492",4,"6,720",700,55,77,"5,998","8,224","14,222"
8,"108,492","16,972,731","100,891","6,475,979","4,577","10,495,906","13,168","10,096","73,093","11,256","93,860","157,080",15,"11,271",885,86,93,"10,379","12,332","22,711"
9,"316,795","16,975,168","300,497","2,397,406","1,717","14,565,452","34,108","9,642","255,394","62,467","286,274","4,092,137","2,113","64,579","1,590","2,115",60,"65,045","20,539","85,583"


### Current Distribution Table

In [7]:
create_distribution_table(calc.records, groupby='weighted_deciles', result_type='weighted_avg')

,expanded_income,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,num_returns_AMT,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined
0,"-6,904","16,222,114","-7,252","11,094,216","5,964","6,150",18,"4,135",44,4,"-7,601",0,0,4,0,6,141,-131,353,222
1,"8,443","16,220,773","6,016","13,627,903","7,057","48,948",29,"4,913",262,23,"5,990","20,950",0,24,0,4,774,-747,821,74
2,"15,025","16,223,499","10,098","13,243,287","7,608","279,753",180,"6,493","1,482",145,"9,974","27,189",1,146,19,6,"1,650","-1,517","1,428",-89
3,"21,963","16,222,202","15,120","14,169,419","7,787","683,754",548,"6,822","3,991",420,"14,767","31,485",2,422,82,13,"1,460","-1,107","2,039",932
4,"30,413","16,223,284","22,440","14,142,144","7,921","1,516,658","1,287","7,187","8,668",999,"21,647","18,616",2,"1,002",217,17,"1,067",-265,"2,881","2,615"
5,"40,936","16,222,555","32,564","13,206,631","7,687","2,863,096","2,661","7,339","16,532","2,017","30,986","15,840",2,"2,020",362,26,548,"1,136","3,965","5,101"
6,"54,706","16,222,223","46,478","11,339,850","7,067","4,849,894","5,036","7,734","27,243","3,573","43,467","10,617",2,"3,575",506,42,164,"2,947","5,336","8,283"
7,"74,668","16,222,702","67,782","9,028,824","6,082","7,185,021","8,156","8,574","45,098","6,519","63,224","46,267",5,"6,524",681,62,80,"5,825","7,708","13,533"
8,"110,558","16,222,104","102,839","5,871,767","4,308","10,349,258","13,730","10,142","74,728","11,702","95,552","184,541",18,"11,720",889,100,95,"10,835","12,479","23,314"
9,"338,763","16,223,164","321,658","2,019,136","1,497","14,190,613","36,479","9,580","274,512","69,165","306,721","4,448,053","2,351","71,516","1,789","2,138",55,"71,811","21,957","93,768"


### Diagnostic Table Comparison

In [8]:
pulp_diag = create_diagnostic_table(calc_pulp)

In [9]:
diag = create_diagnostic_table(calc)

In [10]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['PULP'] = pulp_diag[2014]
diag_data['PUF'] = diag[2014]
diag_data['% Change'] = ((pulp_diag[2014] / diag[2014]) - 1) * 100

In [11]:
diag_data

,SOI,PULP,PUF,% Change
Returns (#m),148.6,169.7,162.2,4.6
AGI ($b),"9,771.00","10,392.2","10,021.5",3.7
Itemizers (#m),44,44.2,42.0,5.3
Itemized Deduction ($b),1206.7,"1,121.9","1,105.2",1.5
Standard Deduction Filers (#m),117.4,113.7,107.7,5.5
Standard Deduction ($b),876.2,"1,005.3",925.2,8.6
Personal Exemption ($b),1121.6,"1,202.7","1,114.0",8.0
Taxable Income ($b),6997.9,"7,493.6","7,341.8",2.1
Regular Tax ($b),NaN,"1,505.2","1,534.2",-1.9
AMT Income ($b),NaN,"9,842.4","9,485.9",3.8


### Income Levels

In [12]:
inc_dict = OrderedDict()
inc_dict['PULP'] = [] 
inc_dict['Current'] = []
inc_dict['Diff'] = []
inc_dict['Pct Diff'] = []
inc_list = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
was_pulp = (calc_pulp.records.e00200 * calc_pulp.records.s006).sum()
inc_dict['PULP'].append(was_pulp)
was = (calc.records.e00200 * calc.records.s006).sum()
inc_dict['Current'].append(was)
was_diff = (was_pulp - was)
inc_dict['Diff'].append(was_diff)
inc_dict['Pct Diff'].append((was_diff / was) * 100)

int_pulp = (calc_pulp.records.e00300 * calc_pulp.records.s006).sum()
inc_dict['PULP'].append(int_pulp)
int_ = (calc.records.e00300 * calc.records.s006).sum()
inc_dict['Current'].append(int_)
int_diff = (int_pulp - int_)
inc_dict['Diff'].append(int_diff)
inc_dict['Pct Diff'].append((int_diff / int_) * 100)

odiv_pulp = (calc_pulp.records.e00600 * calc_pulp.records.s006).sum()
inc_dict['PULP'].append(odiv_pulp)
odiv = (calc.records.e00600 * calc.records.s006).sum()
inc_dict['Current'].append(odiv)
odiv_diff = (odiv_pulp - odiv)
inc_dict['Diff'].append(odiv_diff)
inc_dict['Pct Diff'].append((odiv_diff / odiv) * 100)

qdiv_pulp = (calc_pulp.records.e00650 * calc_pulp.records.s006).sum()
inc_dict['PULP'].append(qdiv_pulp)
qdiv = (calc.records.e00650 * calc.records.s006).sum()
inc_dict['Current'].append(qdiv)
qdiv_diff = (qdiv_pulp - qdiv)
inc_dict['Diff'].append(qdiv_diff)
inc_dict['Pct Diff'].append((qdiv_diff / qdiv) * 100)

biz_pulp = (calc_pulp.records.e00900 * calc_pulp.records.s006).sum()
inc_dict['PULP'].append(biz_pulp)
biz = (calc.records.e00900 * calc.records.s006).sum()
inc_dict['Current'].append(biz)
biz_diff = (biz_pulp - biz)
inc_dict['Diff'].append(biz_diff)
inc_dict['Pct Diff'].append((biz_diff / biz) * 100)

inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = inc_list
inc_df

,PULP,Current,Diff,Pct Diff
WAS,"7,073,136,919,367.5","6,820,912,371,695.9","252,224,547,671.6",3.7
Taxable Interest,"104,610,988,951.6","97,246,196,658.4","7,364,792,293.2",7.6
Ordinary Dividends,"280,092,812,553.8","270,137,086,776.2","9,955,725,777.6",3.7
Qualified Dividends,"206,976,545,606.9","198,124,994,913.5","8,851,550,693.4",4.5
Business Income,"333,505,087,603.8","309,444,223,964.1","24,060,863,639.8",7.8


### Distribution of Income Variables

In [13]:
# Generate data for distribution plots
pulp_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
pulp_was = distribution(calc_pulp.records.e00200, calc_pulp.records.s006, calc_pulp.records.c00100)
cur_was = distribution(calc.records.e00200, calc.records.s006, calc.records.c00100)
pulp_int = distribution(calc_pulp.records.e00300, calc_pulp.records.s006, calc_pulp.records.c00100)
cur_int = distribution(calc.records.e00300, calc.records.s006, calc.records.c00100)
pulp_odiv = distribution(calc_pulp.records.e00600, calc_pulp.records.s006, calc_pulp.records.c00100)
cur_odiv = distribution(calc.records.e00600, calc.records.s006, calc.records.c00100)
pulp_qdiv = distribution(calc_pulp.records.e00650, calc_pulp.records.s006, calc_pulp.records.c00100)
cur_qdiv = distribution(calc.records.e00650, calc.records.s006, calc.records.c00100)
pulp_biz = distribution(calc_pulp.records.e00900, calc_pulp.records.s006, calc_pulp.records.c00100)
cur_biz = distribution(calc.records.e00900, calc.records.s006, calc.records.c00100)
pulp_dist['WAS'] = pulp_was[1]
cur_dist['WAS'] = cur_was[1]
pulp_dist['INT'] = pulp_int[1]
cur_dist['INT'] = cur_int[1]
pulp_dist['ODIV'] = pulp_odiv[1]
cur_dist['ODIV'] = cur_odiv[1]
pulp_dist['QDIV'] = pulp_qdiv[1]
cur_dist['QDIV'] = cur_qdiv[1]
pulp_dist['BIZ'] = pulp_biz[1]
cur_dist['BIZ'] = cur_biz[1]
pulp_dist['AGI Bin'] = index_list()
cur_dist['AGI Bin'] = index_list()
pulp_dist['label'] = 'PULP'
cur_dist['label'] = 'Current'
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum()) * 100
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'
scatter_data = pd.concat([pulp_dist, cur_dist, soi_dist])
scatter_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(scatter_data, x='AGI Bin', y=item[0], color='label', ylabel='Percent',
                      title=title, tooltips=[('PCT', '@{}'.format(item[0]))])
    scatter_list.append(scatter)

In [14]:
show(column(scatter_list))

In [14]:
pulp_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
pulp_tot['WAS'] = pulp_was[0]
cur_tot['WAS'] = cur_was[0]
pulp_tot['INT'] = pulp_int[0]
cur_tot['INT'] = cur_int[0]
pulp_tot['ODIV'] = pulp_odiv[0]
cur_tot['ODIV'] = cur_odiv[0]
pulp_tot['QDIV'] = pulp_qdiv[0]
cur_tot['QDIV'] = cur_qdiv[0]
pulp_tot['BIZ'] = pulp_biz[0]
cur_tot['BIZ'] = cur_biz[0]
pulp_tot['AGI Bin'] = index_list()
cur_tot['AGI Bin'] = index_list()
pulp_tot['label'] = 'PULP'
cur_tot['label'] = 'Current'
soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'
total_data = pd.concat([pulp_tot, cur_tot, soi_income])
# Create scatter plot objects
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
total_list = list()  # list for scatter plot objects
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    scatter = Scatter(total_data, x='AGI Bin', y=item[0], color='label', ylabel='Total',
                      title=title, tooltips=[('Total', '@{}'.format(item[0]))])
    total_list.append(scatter)

In [16]:
show(column(total_list))

### Itemized Deduction Amounts

In [15]:
deductions_pulp = {'Medical Expenses': (calc_pulp.records.e17500[calc_pulp.records.c04470 > 0] *
                                        calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'State and Local Taxes':  (calc_pulp.records.e18400[calc_pulp.records.c04470 > 0] *
                                              calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Real Estate Taxes':  (calc_pulp.records.e18500[calc_pulp.records.c04470 > 0] *
                                          calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Interest Paid':  (calc_pulp.records.e19200[calc_pulp.records.c04470 > 0] *
                                       calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Charitable Cash Contributions': (calc_pulp.records.e19800[calc_pulp.records.c04470 > 0] *
                                                     calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Charitable Non-Cash Contributions': (calc_pulp.records.e20100[calc_pulp.records.c04470 > 0] *
                                                         calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Total Misc. Expenses':  (calc_pulp.records.e20400[calc_pulp.records.c04470 > 0] *
                                             calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum(),
                   'Net Casualty or Loss': (calc_pulp.records.g20500[calc_pulp.records.c04470 > 0] *
                                            calc_pulp.records.s006[calc_pulp.records.c04470 > 0]).sum()}
ded_pulp_df = pd.DataFrame.from_dict(deductions_pulp, 'index')
ded_pulp_df.columns = ['Total']
ded_pulp_df['source'] = 'PULP'

deductions = {'Medical Expenses': (calc.records.e17500[calc.records.c04470 > 0] *
                                   calc.records.s006[calc.records.c04470 > 0]).sum(),
              'State and Local Taxes':  (calc.records.e18400[calc.records.c04470 > 0] *
                                         calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Real Estate Taxes':  (calc.records.e18500[calc.records.c04470 > 0] *
                                     calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Interest Paid':  (calc.records.e19200[calc.records.c04470 > 0] *
                                 calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Cash Contributions': (calc.records.e19800[calc.records.c04470 > 0] *
                                                calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Charitable Non-Cash Contributions': (calc.records.e20100[calc.records.c04470 > 0] *
                                                    calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Total Misc. Expenses':  (calc.records.e20400[calc.records.c04470 > 0] *
                                        calc.records.s006[calc.records.c04470 > 0]).sum(),
              'Net Casualty or Loss': (calc.records.g20500[calc.records.c04470 > 0] *
                                       calc.records.s006[calc.records.c04470 > 0]).sum()}
ded_df = pd.DataFrame.from_dict(deductions, 'index')
ded_df.columns = ['Total']
ded_df['source'] = 'Current'
soi_deductions['source'] = 'SOI'

ded_full_df = pd.concat([ded_pulp_df, ded_df, soi_deductions])
ded_full_df['index'] = ded_full_df.index

In [18]:
ded_bar = Bar(ded_full_df, 'index', 'Total', group='source', title='Itemized Deduction Totals',
              xlabel='Deduction', ylabel='Total', tooltips=[('Deduction', '@index'), ('Total', '@height{0.2f}'),
                                                            ('Data', '@source')])
show(ded_bar)

In [16]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference - PULP'] = ded_pulp_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - PULP'] = 100 * ded_error_df['Difference - PULP'] / soi_deductions['Total']
ded_error_df['Difference - Current'] = ded_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference - Current'] = 100 * ded_error_df['Difference - Current'] / soi_deductions['Total']
print 'Error in Itemized Deductions Relative to SOI Totals'
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference - PULP,% Difference - PULP,Difference - Current,% Difference - Current
Charitable Cash Contributions,"-892,595,932.6",-0.6,"-4,703,427,583.1",-3.0
Charitable Non-Cash Contributions,"-30,613,850,428.2",-46.9,"-31,555,102,263.5",-48.3
Interest Paid,"-129,506,731.4",-0.0,"-7,489,448,194.8",-2.4
Medical Expenses,"-2,703,736,680.1",-2.1,"-9,779,156,210.6",-7.6
Net Casualty or Loss,"2,278,869,831.7",103.4,"2,251,393,716.6",102.1
Real Estate Taxes,"20,647,291,268.0",11.4,"17,247,882,146.1",9.5
State and Local Taxes,"-27,961,525,237.3",-8.5,"-16,248,434,290.2",-4.9
Total Misc. Expenses,"11,121,014,725.6",8.8,"3,957,677,605.4",3.1


In [17]:
# Create DataFrame subset of calculator records for the PULP weights
item_df_pulp = pd.DataFrame({'e00200': calc_pulp.records.e00200,
                             's006': calc_pulp.records.s006,
                             'c04470': calc_pulp.records.c04470})
item_df_pulp['itemizer'] = np.where(calc_pulp.records.c04470 > 0, 1, 0)
item_participation_rt_pulp = percentile(item_df_pulp, 'itemizer', 100, 'e00200', 's006')
# Create DataFrame subset for current weights
item_df = pd.DataFrame({'e00200': calc.records.e00200,
                        's006': calc.records.s006,
                        'c04470': calc.records.c04470})
item_df['itemizer'] = np.where(calc.records.c04470 > 0, 1, 0)
item_participation_rt = percentile(item_df, 'itemizer', 100, 'e00200', 's006')
item_index = item_participation_rt_pulp.index

In [21]:
hover_ded = HoverTool(tooltips=[('Percentile', '$index'), ('Itemizing Rate', '$x')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile', tools=[hover_ded, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_participation_rt_pulp, legend='PULP')
f.line(item_index, item_participation_rt, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)

In [18]:
# Average Itemized Deduction by percentile
itemizers_pulp = copy.deepcopy(item_df_pulp[item_df_pulp['c04470'] > 0])
item_mean_pulp = percentile(itemizers_pulp, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(item_df[item_df['c04470'] > 0])
item_mean = percentile(itemizers, 'c04470', 100, 'e00200', 's006')

In [23]:
hover = HoverTool(tooltips=[('Percentile', '$index'), ('Mean', '$y')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile', tools=[hover, 'save', 'reset', 'zoom_in', 'zoom_out'])
f.line(item_index, item_mean_pulp, legend='PULP')
f.line(item_index, item_mean, legend='Current', color='red')
f.legend.location = 'top_left'
show(f)

#### Weight Comparison

In [133]:
wt_comp = pd.DataFrame()
wt_comp['AGI'] = calc.records.c00100
wt_comp['pulp wt'] = calc_pulp.records.s006
wt_comp['current wt'] = calc.records.s006
wt_comp['Wage'] = calc.records.e00200
wt_comp = wt_comp.sort_values(by='AGI')
wt_comp['s006'] = calc.records.s006

In [134]:
wt_comp = add_weighted_income_bins(wt_comp,15, income_measure='AGI')
gpdf = wt_comp.groupby('bins', as_index=False)

In [135]:
gpdf[['pulp wt', 'current wt', 'AGI', 'Wage']].mean()

,bins,pulp wt,current wt,AGI,Wage
0,1,"1,011.6","1,076.6","-585,015.9","39,881.3"
1,2,"1,507.3","1,759.7",849.3,937.0
2,3,"1,059.9","1,236.7","3,981.2","3,180.9"
3,4,"1,038.9","1,037.1","7,774.5","5,669.8"
4,5,"1,038.4",998.2,"12,006.3","8,518.1"
5,6,"1,021.3","1,036.7","16,471.7","12,251.3"
6,7,"1,001.0",859.3,"21,925.7","17,695.6"
7,8,984.4,907.0,"28,452.9","23,439.7"
8,9,981.8,859.1,"36,043.7","29,919.2"
9,10,959.9,816.8,"45,392.7","36,996.8"


In [136]:
gpdf[['pulp wt', 'current wt', 'AGI', 'Wage']].sum()

,bins,pulp wt,current wt,AGI,Wage
0,1,"10,161,535.5","10,814,553.1","-5,876,484,259.5","400,607,890.9"
1,2,"9,263,858.7","10,815,273.7","5,219,844.4","5,758,986.7"
2,3,"9,268,977.7","10,815,082.6","34,815,461.5","27,817,131.4"
3,4,"10,834,035.0","10,814,816.0","81,072,786.0","59,124,241.4"
4,5,"11,250,256.6","10,815,028.1","130,076,610.8","92,285,079.4"
5,6,"10,653,150.2","10,814,270.0","171,816,140.0","127,793,468.0"
6,7,"12,599,037.8","10,815,629.8","275,957,433.2","222,716,320.0"
7,8,"11,738,543.6","10,815,053.6","339,272,018.7","279,495,441.8"
8,9,"12,358,592.0","10,814,757.9","453,718,721.4","376,623,452.9"
9,10,"12,708,797.2","10,814,183.9","600,999,584.1","489,837,164.6"
